# Problem 1

In [1]:
%pylab inline 
%matplotlib inline
import random

Populating the interactive namespace from numpy and matplotlib


If one wanted to solve a quadratic of the form $ax^2 +bx + c$, we would turn to our old tried and trusted friend, the quadratic formula, giving the roots to be $$x_1 = \frac{-b + \sqrt{b^2 -4ac}}{2a}, \quad x_2 = \frac{-b - \sqrt{b^2 -4ac}}{2a}.$$

However, the nature of life is such that this approach does not always work when using a computer, as we will see below.

In [2]:
def quadraticformula(a,b,c):
    '''
    computes roots x1 and x2 from a, b, and c using
    the classical quadratic formula
    '''
    #discriminant
    disc = (b**2 - 4.0*a*c)
    
    #solutions can be split into three classes:
    #complex conjugate pairs
    #double roots
    #two distinct real roots

    if(disc<0):
        print("No real solutions.")
        return complex(-b/(2.*a),sqrt(-disc)/(2.*a)),complex(-b/(2.*a),-sqrt(-disc)/(2.*a))
    
    elif(disc==0):
        print("One root of multiplicity 2.")
        x = -b/(2.*a)
        return x,x
    
    else:
        print("Two distinct roots.")
        x1 = (-b + np.sqrt(disc)) / (2.*a)
        x2 = (-b - np.sqrt(disc)) / (2.*a)
        return x1,x2

We now test the above implementation, and show how (and why) it can break.

Note: it is assumed that the coefficients of the quadratic are real

We start with a quadratic whose roots are nice numbers - the golden ratio $\phi = \frac{1 + \sqrt{5}}{2}$ and its conjugate $\frac{-1}{\phi} = \frac{1 - \sqrt{5}}{2}$.

In [3]:
a=1.0
b=-1.0
c=-1.0
X1,X2 = quadraticformula(a,b,c)
print("Roots:",X1,X2)
print("Errors in roots:",abs(a*X1*X1 + b*X1 +c),abs(a*X2*X2 + b*X2 +c))

Two distinct roots.
Roots: 1.618033988749895 -0.6180339887498949
Errors in roots: 0.0 0.0


### Write more test cases for this, and show when it breaks
a = 1.0 b = -1e8 c = 1.0 is an example of when cancellation error will creep in

In [4]:
def test():
    a = random.random()
    b = random.random()
    c = random.random()
    d = random.random()
    e = random.random()
    f = random.random()
    g = random.random()
    h = random.random()
    i = random.random()
    x1,x2 = quadraticformula(a,b,c)
    x3,x4 = quadraticformula(d,e,f)
    x5,x6 = quadraticformula(g,h,i)
    atol = 1.0e-10
    if (abs(a*x1*x1 + b*x1 +c)<=atol) and (abs(a*x2*x2 + b*x2 +c)<=atol):
        print("Test 1 Passed")
        t1 = 1
    else:
        print("Test 1 Failed")
        t1 = 0
    if (abs(d*x3*x3 + e*x3 +f)<=atol) and (abs(d*x4*x4 + e*x4 +f)<=atol):
        print("Test 2 Passed")
        t2 = 1
    else:
        print("Test 2 Failed")
        t2 = 0
    if (abs(g*x5*x5 + h*x5 +i)<=atol) and (abs(g*x6*x6 + h*x6 +i)<=atol):
        print("Test 3 Passed")
        t3 = 1
    else:
        print("Test 3 Failed")
        t3 = 0
    
    if t1*t2*t3:
        print("Pass")
    else:
        print("Fail")
    return

In [5]:
test()

Two distinct roots.
No real solutions.
Two distinct roots.
Test 1 Passed
Test 2 Passed
Test 3 Passed
Pass


In [6]:
test()

No real solutions.
No real solutions.
No real solutions.
Test 1 Passed
Test 2 Passed
Test 3 Passed
Pass


In [7]:
test()

No real solutions.
Two distinct roots.
No real solutions.
Test 1 Passed
Test 2 Passed
Test 3 Passed
Pass


So far this looks good. But let's see how this implementation can break.

In [8]:
a = 1.0
b = -1.0e8
c = 1.0
X1,X2 = quadraticformula(a,b,c)
print("Roots:",X1,X2)
refs = np.roots([a,b,c]) #comparing with reference solutions
print("Errors in roots:",abs(X1-refs[0]),abs(X2-refs[1]))

Two distinct roots.
Roots: 100000000.0 7.450580596923828e-09
Errors in roots: 1.4901161193847656e-08 2.5494194030761737e-09


This doesn't look that great anymore. We can definitely do much better. What happened? We are running into cancellation error due to the term $b$ dominating the discriminant. Let's look for a work around.

### From the quadratic formula, if we multiply throughout by the "conjugate" of the radical expression in the numerator, we can see that the roots of the given quadratic can be given by: 

$$x_1 = \frac{2c}{-b + \sqrt{b^2 -4ac}}, \quad x_2 = \frac{2c}{-b - \sqrt{b^2 -4ac}}.$$

The formula above is useful when we want to avoid cancelation errors.

In [9]:
def quadratic_alternative(a,b,c):
    '''
    computes roots x1 and x2 from a, b, and c using the alternative forms for the roots
    these expressions come from a sort of inverse rationalization process
    '''
    disc = (b**2 - 4.*a*c)
    if(disc<0):
        print("No real solutions.")
        return
    elif(disc==0):
        print("One root of multiplicity 2.")
        x = -(2*c)/b
        return x,x
    else:
        print("Two distinct roots.")
        x1 = 2*c / (-b - np.sqrt(disc))
        x2 = 2*c / (-b + np.sqrt(disc))
        return x1, x2

In [10]:
a = 1.0
b = -1.0e8
c = 1.0
X1,X2 = quadratic_alternative(a,b,c)
print("Roots:",X1,X2)
refs = np.roots([a,b,c]) #comparing with reference solutions
print("Errors in roots:",abs(X1-refs[0]),abs(X2-refs[1]))

Two distinct roots.
Roots: 134217728.0 1e-08
Errors in roots: 34217728.000000015 1.6543612251060553e-24


So we see that our alternative implementation now gives us a much better error handle for one of the roots, but fails miserably for the other. We seek a solution that will do a balancing act and give us the best of both worlds, or rather, formulas.

## Final Function
The final function quadratic roots performs the above operations and analyzes which formula to use

In [11]:
def quadratic_roots(a,b,c):
    '''
    Computes the two roots x1 and x2 of the quadratic equation a*x^2 + b*x + c = 0.
    This code will address the problem of cancellation and overflow errors.
    '''

    if a==0 and b==0 and c==0:
        print("Infinitely many solutions - trivial equation 0 = 0.")
        return ("N/A","N/A")
    
    # easiest way to handle errors due to large coefficients to scale the numbers to lie between 0 and 1
    m = max(abs(a),abs(b),abs(c))
    a /= m
    b /= m
    c /= m

    # dealing with various input classes
    if a == 0: # equation becomes bx + c = 0
        if b == 0: # equation becomes c = 0
            print("Equation has no solutions")
            return ("NaN","NaN")
        else: # equation is bx+c = 0
            print("The equation is linear and only has one solution")
            return (- c/b,-c/b)
    elif c == 0: # Equation becomes ax^2 + bx = 0
        print("The equation has two solutions.")
        x1 = 0
        x2 = -b/a 
        return (x1,x2)
    else: # ax^2+bx+c=0
        print("The equation has two solutions.")
        disc = sqrt(b**2.0 - 4.0*a*c)
        if(disc<0):
            print("No real solutions.")
            return complex(-b/2.*a,sqrt(-disc)/2.*a),complex(-b/2.*a,-sqrt(-disc)/2.*a)
    
        elif(disc==0):
            print("One root of multiplicity 2.")
            x = -b/(2.*a)
            return (x,x)
        else:
            x1 = (-b - np.sign(b)*disc)/(2.0*a) #the sign function helps takes care of potential cancellation error
            x2 = (c)/(a*x1)
            return (x1,x2)

In [12]:
x1,x2 = quadratic_roots(0.0,0.0,0.0)

Infinitely many solutions - trivial equation 0 = 0.


In [13]:
a = 1.0
b = -1.0e8
c = 1.0
X1,X2 = quadratic_roots(a,b,c)
print("Roots:",X1,X2)
refs = np.roots([a,b,c]) #comparing with reference solutions
print("Errors in roots:",abs(X1-refs[0]),abs(X2-refs[1]))

The equation has two solutions.
Roots: 99999999.99999999 1.0000000000000002e-08
Errors in roots: 0.0 0.0


In [14]:
a = 1.0
b = 1e5
c = 1.0
X1,X2 = quadratic_roots(a,b,c)
print("Roots:",X1,X2)
refs = np.roots([a,b,c]) #comparing with reference solutions
print("Errors in roots:",abs(X1-refs[0]),abs(X2-refs[1]))

The equation has two solutions.
Roots: -99999.99999 -1.0000000001e-05
Errors in roots: 0.0 1.6940658945086007e-21


In [15]:
a = 6e154
b = 5e154
c = -4e154
X1,X2 = quadratic_roots(a,b,c)
print("Roots:",X1,X2)
refs = np.roots([a,b,c]) #comparing with reference solutions
print("Errors in roots:",abs(X1-refs[0]),abs(X2-refs[1]))

The equation has two solutions.
Roots: -1.3333333333333333 0.5
Errors in roots: 0.0 0.0


The above equation should give us the same roots as $(a,b,c) = (6,5,-4)$, lets check.

In [16]:
a = 6
b = 5
c = -4
X1,X2 = quadratic_roots(a,b,c)
print("Roots:",X1,X2)
refs = np.roots([a,b,c]) #comparing with reference solutions
print("Errors in roots:",abs(X1-refs[0]),abs(X2-refs[1]))

The equation has two solutions.
Roots: -1.3333333333333333 0.5
Errors in roots: 0.0 0.0


### This function works well, and is giving us results on par with python's (numpy's) root-finding routine. 